In [13]:
import csv
import math

import numpy as np

### Tensorflow
import tensorflow as tf
from tensorflow.contrib import slim

In [14]:
INPUT_SIZE = 28
SPLIT_SIZE = 4

### train_x = train_x / NORMALIZE
NORMALIZE = 1.0
# NORMALIZE = 255   

In [15]:
### Dataset class
class Dataset(object) :
    
    def __init__(self, path, batch_size, shuffle = True, normalize = 255.0) :
        self.batch_size = batch_size
        self.shuffle    = shuffle
        self.normalize  = normalize
        self.xs, self.ys = self._load_data(path)
        
        self.data_size = self.xs.shape[0]
        self.batch_max = int(math.ceil(self.data_size / float(self.batch_size)))
        
    def _load_data(self, path) :
        reader = csv.reader(open(path), lineterminator = "\n")
        data   = []
        labels = []
        for row in reader :
            tmp = np.asarray([float(x) for x in row])
            
            ### Normalize
            tmp = tmp / self.normalize
            
            data.append(tmp[:INPUT_SIZE*INPUT_SIZE])
            labels.append(tmp[INPUT_SIZE*INPUT_SIZE:])
        
        data   = np.asarray(data)
        labels = np.asarray(labels)
        return (data, labels)

    def __call__(self) :
        indexes = None
        if self.shuffle :
            indexes = np.random.permutation(self.data_size)
        else :
            indexes = np.arange(self.data_size)

        for d_idx in range(self.batch_max) :
            start =     (d_idx + 0) * self.batch_size
            end   = min((d_idx + 1) * self.batch_size, self.data_size)
            
            batch_data_size = len(indexes[start:end])
            xs = np.zeros((batch_data_size, self.xs.shape[1]), dtype = np.float32)
            ys = np.zeros((batch_data_size, self.ys.shape[1]), dtype = np.float32)
            
            for b_idx, r_idx in enumerate(indexes[start:end]) :
                xs[b_idx] = self.xs[r_idx]
                ys[b_idx] = self.ys[r_idx]

            yield (xs, ys)

In [16]:
### Load dataset
dataset = Dataset("training_data_4x4.csv", 10, shuffle = True, normalize = NORMALIZE)

In [17]:
valid_dataset = Dataset("training_data_4x4.csv", 10, shuffle = False, normalize = NORMALIZE)

In [18]:
### Load batch data sample code
# for batch_num, (train_x, train_y) in enumerate(dataset(), 1) :
#     print("Batch {0}".format(batch_num))
#     print("Train shape : {0}".format(train_x.shape))

In [19]:
### for jupyter, interactive session.
sess = tf.InteractiveSession()

In [20]:
### Network settings
NN_INPUT_SIZE  = INPUT_SIZE * INPUT_SIZE
NN_OUTPUT_SIZE = 4 * SPLIT_SIZE

### Input and Output
x  = tf.placeholder(tf.float32, shape=[None, NN_INPUT_SIZE], name = 'data')
y_ = tf.placeholder(tf.float32, shape=[None, NN_OUTPUT_SIZE], name = 'label')
is_training = tf.placeholder(tf.bool, [])

def print_shape(layer) :
    print("Layer \"{0}\" shape : {1}".format(layer.name, layer.shape))

with slim.arg_scope([slim.conv2d, slim.fully_connected], 
                    activation_fn=tf.nn.relu, 
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                    biases_initializer=tf.constant_initializer(0.1),
                    normalizer_fn = slim.batch_norm,
                    normalizer_params = {'is_training': is_training},
                   ), \
    slim.arg_scope([slim.max_pool2d], padding='SAME'), \
    slim.arg_scope([slim.conv2d_transpose], 
                   padding='SAME', 
                   activation_fn = None, 
                   stride = 2, 
                   weights_initializer = tf.contrib.layers.xavier_initializer(),
                   normalizer_fn = slim.batch_norm,
                   normalizer_params = {'is_training': is_training}
                  ) :
        
        x_image = slim.array_ops.reshape(x, [-1, 28, 28, 1])
        print_shape(x_image)
        
        ### Convolution
        h_conv1 = slim.conv2d(x_image, 32, [3, 3])
        h_pool1 = slim.max_pool2d(h_conv1, [2, 2])
        print_shape(h_conv1)
        print_shape(h_pool1)

        h_conv2 = slim.conv2d(h_pool1, 64, [3, 3])
        h_pool2 = slim.max_pool2d(h_conv2, [2, 2])
        print_shape(h_conv2)
        print_shape(h_pool2)

        h_conv3 = slim.conv2d(h_pool2, 64, [3, 3])
        h_pool3 = slim.max_pool2d(h_conv3, [2, 2])
        print_shape(h_conv3)
        print_shape(h_pool3)  

        ### Global (fully connect)
        h_pool3_flat = slim.flatten(h_pool3)
        print_shape(h_pool3_flat)
        h_pool3_fc   = slim.fully_connected(h_pool3_flat, NN_OUTPUT_SIZE, activation_fn = None, normalizer_fn = None)
        h_pool3_reconstract = slim.array_ops.reshape(h_pool3_fc, [-1, 4, SPLIT_SIZE, 1])
        print_shape(h_pool3_reconstract)
        
        ### Local (Convolution)
        h_pool3_conv = slim.conv2d(h_pool3, 1, [3, 3], activation_fn = None, normalizer_fn = None)
        print_shape(h_pool3_conv)
        
        ### Add global and local
        h_add = tf.add(h_pool3_reconstract, h_pool3_conv)
        y_flat = slim.flatten(h_add)
        print_shape(y_flat)

Layer "Reshape_2:0" shape : (?, 28, 28, 1)
Layer "Conv_4/Relu:0" shape : (?, 28, 28, 32)
Layer "MaxPool2D_3/MaxPool:0" shape : (?, 14, 14, 32)
Layer "Conv_5/Relu:0" shape : (?, 14, 14, 64)
Layer "MaxPool2D_4/MaxPool:0" shape : (?, 7, 7, 64)
Layer "Conv_6/Relu:0" shape : (?, 7, 7, 64)
Layer "MaxPool2D_5/MaxPool:0" shape : (?, 4, 4, 64)
Layer "Flatten_2/Reshape:0" shape : (?, 1024)
Layer "Reshape_3:0" shape : (?, 4, 4, 1)
Layer "Conv_7/BiasAdd:0" shape : (?, 4, 4, 1)
Layer "Flatten_3/Reshape:0" shape : (?, 16)


In [21]:
### Loss
loss = tf.losses.mean_squared_error(y_, y_flat)

In [22]:
### Training step (Optimizer)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [23]:
### Initialize session
sess.run(tf.global_variables_initializer())

In [24]:
### Training
EPOCH = 10000
VALIDATION_EPOCH = 1000
for epoch in range(1, EPOCH + 1) :
    
    train_loss  = 0
    train_count = 0
    for train_x, train_y in dataset() :
        train_count += 1
        _, tmp_loss = sess.run([train_step, loss], feed_dict = {x: train_x, y_: train_y, is_training: True})
        batch_loss = tmp_loss / train_x.shape[0]
        train_loss += batch_loss
    if train_count > 0 :
        train_loss /= train_count

    ### Validation
    if epoch % VALIDATION_EPOCH == 0 :
        valid_loss  = 0
        valid_count = 0
        for (valid_x, valid_y) in valid_dataset() :
            valid_count += 1
            tmp_loss   = sess.run(loss, feed_dict = {x: valid_x, y_: valid_y, is_training: True}) # TODO (※)
            batch_loss = tmp_loss / valid_x.shape[0]
            valid_loss += batch_loss
        if valid_count > 0 :
            valid_loss /= valid_count
        print("Epoch {0} train loss = {1}, valid loss = {2}, sqrt(valid loss) * {4} = {3}".format(epoch, train_loss, valid_loss, math.sqrt(valid_loss) * NORMALIZE, NORMALIZE))

# (※) 正しくは「is_training : False」とすべき。
#      本来は valid のために batch norm 用に値を保持する必要があるが、本プログラムでは保持していないため、is_training モードが正しく動作しない。
#      train データと valid データそれぞれの平均・分散値が近いという仮定から、ここでは保持せず、valid 時も batch norm そのまま利用している。

Epoch 1000 train loss = 45.099233572823664, valid loss = 57.040126255580354, sqrt(valid loss) * 1.0 = 7.5524913939428195
Epoch 2000 train loss = 3.4401085444859096, valid loss = 7.684740774972099, sqrt(valid loss) * 1.0 = 2.7721365000612974
Epoch 3000 train loss = 1.8087670734950474, valid loss = 3.4743043490818573, sqrt(valid loss) * 1.0 = 1.86394859078298
Epoch 4000 train loss = 0.817995994431632, valid loss = 2.2420745304652625, sqrt(valid loss) * 1.0 = 1.4973558463055008
Epoch 5000 train loss = 1.2049815927233014, valid loss = 1.5402809517724172, sqrt(valid loss) * 1.0 = 1.24108055813167
Epoch 6000 train loss = 0.45289166825158256, valid loss = 1.0800246034349714, sqrt(valid loss) * 1.0 = 1.0392423218070805
Epoch 7000 train loss = 0.5516463620322092, valid loss = 1.0291009902954102, sqrt(valid loss) * 1.0 = 1.0144461495295896
Epoch 8000 train loss = 0.33803885664258687, valid loss = 0.720585857118879, sqrt(valid loss) * 1.0 = 0.8488732868449089
Epoch 9000 train loss = 0.20474199567

In [31]:
### Save session
saver = tf.train.Saver()
saver.save(sess, "cae_model.ckpt")

'cae_model.ckpt'

In [32]:
### Load session
saver.restore(sess, "cae_model.ckpt")

INFO:tensorflow:Restoring parameters from cae_model.ckpt
